<a href="https://colab.research.google.com/github/ChhaSahil/Boise_SmartWatch/blob/main/Flipkart_SmartWatches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Noise and boAt smartwatches data Web Scraping from Flipkart

In [ ]:
!pip install selenium

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

### Setting up webdriver

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')

'C:\Users\sahil\Downloads\chromedriver_win32 (1)\chromedriver.exe' is the path of chromedriver executer

In [ ]:
driver = webdriver.Chrome(r"C:\Users\sahil\Downloads\chromedriver_win32 (1)\chromedriver.exe" ,options=options)

In [ ]:
url = 'https://www.flipkart.com/search?q=smartwatch&sid=ajy%2Cbuh&as=on&as-show=on&otracker=AS_QueryStore_HistoryAutoSuggest_1_3_na_na_na&otracker1=AS_QueryStore_HistoryAutoSuggest_1_3_na_na_na&as-pos=1&as-type=HISTORY&suggestionId=smartwatch%7CSmart+Watches&requestId=db7804e9-f819-4fc6-9c90-5a0a64c270c3&as-backfill=on&p%5B%5D=facets.brand%255B%255D%3DboAt&p%5B%5D=facets.brand%255B%255D%3DNoise&p%5B%5D=facets.availability%255B%255D%3DExclude%2BOut%2Bof%2BStock&page='

### To Store the data readily available on searching the smartwatches and also storing watches respective urls

In [ ]:
name = []
price = []
prod_url = []
img = []
for i in range(1,7):
    driver.get(url+str(i))
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,'lxml')
    prod_info = soup.find_all('a',class_ = 'IRpwTa')
    prod_price = soup.find_all('div',class_='_30jeq3')
    img_url = soup.find_all('img',class_='_2r_T1I')
    print(len(name),len(prod_url),len(price),len(img))
    for j in range(len(prod_info)):
        name.append(prod_info[j]['title'])
        prod_url.append(prod_info[j]['href'])
        img.append(img_url[j]['src'])
        price.append(prod_price[j].text)

In [ ]:
dic = {
    'Name':name,
    'Price':price,
    'img_url':img,
    'prod_url':prod_url
}

In [ ]:
data1 = pd.DataFrame(dic)
data1

### Scraping different features of the product

In [ ]:
features = {'Star':[],'Rating':[],'Amoled':[],'Display Resolution':[],'Water Resistant':[],'Battery Life':[],'Charge Time':[],'Alarm Clock':[],'Calendar':[],'Step Count':[],'Calorie Count':[],'Heart Rate Monitor':[],'Voice Control':[],'Call Features':[],'Operating Range':[],'Display Size':[],'Gesture Control':[]}

In [ ]:
for i in range(len(prod_url)):
    driver.get('https://www.flipkart.com'+prod_url[i])
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,'lxml')
    star = soup.find_all('div',class_='_3LWZlK') #Star Rating
    rat = soup.find_all('span',class_="_2_R_DZ") # Total Ratings indicating the customer engagement
#   dis = soup.find_all('li',class_="_21Ahn-")
    row = soup.find_all('td',class_='_1hKmbr') # Feature Description
    ans = soup.find_all('li',class_='_21lJbe') # Feature Response
    features['Star'].append(float(t[0].text))
    try:
        features['Rating'].append(int(rat[0].text.split(' ')[0].replace(',','')))
    except:
        features['Rating'].append(0)
    if 'AMOLED' in name[i]:
        features['Amoled'].append('Yes')
    else:
        features['Amoled'].append('No')
    dic = {}
    for j in range(len(row)):
        dic[row[j].text] = ans[j].text
    for j in features.items():
        if j[0]=='Star'or j[0]=='Amoled' or j[0]=='Rating':
            continue
        if j[0] in dic:

            features[j[0]].append(dic[j[0]])
        else:
            features[j[0]].append('None')

    print(i)

In [ ]:
data2 = pd.DataFrame(features)

### Storing ratings based on features

In [ ]:
ratings = {'5☆_Rating':[],'4☆_Rating':[],'3☆_Rating':[],'2☆_Rating':[],'1☆_Rating':[]}

In [ ]:
features = {'Battery & Charger':[],'Display':[],'Design':[],'Activity Tracking':[]}

In [ ]:
for i in range(len(prod_url)):
    driver.get('https://www.flipkart.com'+prod_url[i])
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,'lxml')
    r = soup.find_all('div',class_='_1uJVNT')
    f = soup.find_all('text',class_='_2Ix0io')
    try:
        ratings['5☆_Rating'].append(r[0].text)
    except:
        ratings['5☆_Rating'].append(0)
    try:
        ratings['4☆_Rating'].append(r[1].text)
    except:
        ratings['4☆_Rating'].append(0)
    try:
        ratings['3☆_Rating'].append(r[2].text)
    except:
        ratings['3☆_Rating'].append(0)
    try:
        ratings['2☆_Rating'].append(r[3].text)
    except:
        ratings['2☆_Rating'].append(0)
    try:
        ratings['1☆_Rating'].append(r[4].text)
    except:
        ratings['1☆_Rating'].append(0)
    try:
        features['Battery & Charger'].append(f[0].text)
    except:
        features['Battery & Charger'].append(0)
    try:
        features['Display'].append(f[1].text)
    except:
        features['Display'].append(0)
    try:
        features['Design'].append(f[2].text)
    except:
        features['Design'].append(0)
    try:
        features['Activity Tracking'].append(f[3].text)
    except:
        features['Activity Tracking'].append(0)
    print(i)


In [ ]:
df = pd.DataFrame(ratings)
df2 = pd.DataFrame(features)

In [ ]:
result1 = pd.concat([data1,data2,df,df2],axis=1,join='inner')

### Information about Dial and Straps

In [ ]:
dial = {'Dial Color':[],'Dial Shape':[],'Strap Color':[],'Strap Material':[]}

In [ ]:
for i in range(0,len(prod_url)):
    driver.get('https://www.flipkart.com'+prod_url[i])
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,'lxml')
    row = soup.find_all('td',class_='_1hKmbr')
    ans = soup.find_all('li',class_='_21lJbe')
    dic = {}
    for j in range(len(row)):
        dic[row[j].text] = ans[j].text
    for j in dial.items():
        if j[0] in dic:

            dial[j[0]].append(dic[j[0]])
        else:
            dial[j[0]].append('None')
    print(i)

In [ ]:
dial_df = pd.DataFrame(dial)

In [ ]:
dial_df

In [ ]:
result_f = pd.concat([result1,dial_df],axis=1,join='inner')

### Data Processing

In [ ]:
# Converting display size from inch to mm
def change_display(s):
    if 'inch' in s or 'Inch' in s:
        p=s.split(' ')
        return float(p[0])*25.4
    else:
        return s.split(' ')[0]

In [ ]:
# Converting Battery life to numeric form
def change_battery_life(s):
    d=''
    for char in s:
        if char.isnumeric():
            d+=char
    return d

In [ ]:
result_f['Display_Size'] = result_f['Display Size'].apply(change_display)
result_f.drop(['Display Size'],axis=1,inplace=True)

In [ ]:
result_f['Battery Life'] = result_f['Battery Life'].apply(change_battery_life)

### Converting Dataframe to CSV file

In [ ]:
result_f.to_csv('output.csv')